In [1]:
import sqlite3
import db
import pandas as pd
import dbga
from collections import defaultdict as dd
import json

conn = sqlite3.connect('databases/test.db')
curs = conn.cursor()

data = pd.read_csv('nyc_rest_health_inspections.csv')
data.head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION
0,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,06/15/2015,Violations were cited in the following area(s).,02B,Hot food item not held at or above 140Âº F.
1,40511702,NOTARO RESTAURANT,MANHATTAN,635,SECOND AVENUE,10016.0,2126863400,Italian,11/25/2014,Violations were cited in the following area(s).,20F,Current letter grade card not posted.


---
1) What is the most commonly cited violation?

Return a single row containing:

- violation code
- violation description
- number of violations

In [2]:
sql1 = '''
SELECT [violation code], [violation description], count(*) as NumViol
FROM test_table
GROUP BY [violation code]
ORDER BY NumViol DESC
LIMIT 1
;'''
pd.read_sql(sql1, conn)

,VIOLATION CODE,VIOLATION DESCRIPTION,NumViol
0,10F,Non-food contact surface improperly constructe...,55772


---

2) How many total violations have occurred for each of the different cuisine types? Limit your results to cuisine types with more than 10,000 violations.

Return:

- cuisine description
- number of violations

Order the results with the most violations on top.

In [3]:
sql2 = '''
SELECT [cuisine description], count(*) as NumViol
FROM test_table               
GROUP BY [cuisine description]
HAVING NumViol > 10000
ORDER BY NumViol DESC
;'''
pd.read_sql(sql2, conn)

,CUISINE DESCRIPTION,NumViol
0,American,89465
1,Chinese,41785
2,"Latin (Cuban, Dominican, Puerto Rican, South &...",18853
3,Pizza,18524
4,Italian,17514
5,CafÃ©/Coffee/Tea,15246
6,Mexican,15006
7,Japanese,14331
8,Caribbean,12826
9,Bakery,11693


---

3) First determine which boro has the most violations. Then determine the restaurant (camis) with the most violations in that boro.

DO NOT specify the camis or boro anywhere in your query, and do not pass in any parameters to the query. The query should still return an up-to-date answer if the data were updated.

Return a single record containing:

- boro
- camis
- dba
- number of violations

In [4]:
sql3 = '''
WITH MaxViolBoro as
  (
   SELECT boro, max(NumViol)
   FROM (
      SELECT boro, count(camis) as NumViol
      FROM test_table
      GROUP BY boro)
  ),

TotalViolByCamis as 
    (
    SELECT camis, count([violation code]) as NumViol, dba, boro
    FROM test_table
    GROUP BY camis
    )

SELECT camis, boro, dba, max(NumViol) as NumViol
FROM TotalViolByCamis
WHERE boro LIKE (SELECT boro FROM MaxViolBoro)
;'''
pd.read_sql(sql3, conn)

,camis,boro,dba,NumViol
0,41459659,MANHATTAN,T. K. KITCHEN,90


---

4) Which restaurant (by their name, i.e. DBA) has the most locations (i.e., distinct camis ids) in each boro, and how many total violations do they have within that boro?

Query should return a total of 5 records (one for each boro) with:

- boro
- dba
- number of locations (in that boro)
- number of violations (in that boro)
- average number of violations per location (i.e., number of violations/number of locations)

Hints:
- You can GROUP BY more than one field at a time
- Make use of the SQL max() function
- Watch out for integer division! The last field (the average) is most likely not an integer

In [8]:
sql4 = ''' 
SELECT dba, boro, max(num_loc) as num_loc, num_viol, 1.0*num_viol/num_loc as avg_viols
FROM (
    SELECT dba, boro, count(DISTINCT(camis)) as num_loc, count(camis) as num_viol
    FROM test_table
    GROUP BY dba, boro
     )           
GROUP BY boro
;'''
pd.read_sql(sql4, conn)

,dba,boro,num_loc,num_viol,avg_viols
0,DUNKIN' DONUTS,BRONX,65,717,11.030769
1,DUNKIN' DONUTS,BROOKLYN,94,785,8.351064
2,STARBUCKS,MANHATTAN,204,1534,7.519608
3,DUNKIN' DONUTS,QUEENS,121,1349,11.148760
4,DUNKIN' DONUTS,STATEN ISLAND,32,415,12.968750


---

5) Get a list of all restaurants that have ever been shut down (the field ACTION will contain the word 'closed'). If a restaurant has been shut down more than once, it should only appear once in the results.

Return:

- camis
- dba
- boro

Order the results by camis, in ascending order.

In [7]:
sql5 = '''
SELECT DISTINCT(camis), dba, boro
FROM test_table
WHERE action LIKE '%closed%'
ORDER BY camis
;'''
pd.read_sql(sql5, conn)

,CAMIS,DBA,BORO
0,40364179,SPOONBREAD TOO,MANHATTAN
1,40365075,MCSORLEY'S OLD ALE HOUSE,MANHATTAN
2,40365226,NOSTRAND DONUT SHOP,BROOKLYN
3,40365472,SPAIN RESTAURANT & BAR,MANHATTAN
4,40365546,REBEN LUNCHEONETTE,BROOKLYN
...,...,...,...
1640,50066625,CROWN FRIED CHICKEN PIZZA,BROOKLYN
1641,50066787,GIOVANNI'S,BRONX
1642,50066879,HK CAFE,QUEENS
1643,50067143,EL FOGON CHARCOAL GRILL,BROOKLYN


In [99]:
conn.close()